# Demographics
- Set up demographic CSV

In [8]:
gsam_prompt = """
The person comes across a stranger who is sitting slumped in a doorway head down, eyes closed, not moving. 
The person hears the stranger coughing and groaning.
"""
q = """
Please provide a value from 0-100 on the person’s inclination to help the stranger. 
0 indicates the person is not at all likely to help to stranger, while 100 indicates the person is very likely to help the stranger.

If any, provide a value for how many minutes the person stays and helps the stranger. 

If any, write a brief story about what actions the person takes.

Please format your answer like so:
Helpfulness: x
Minutes waited: y
Actions:
- Person does/does not z.
"""

In [11]:
import itertools
import pandas as pd

# mandatory persona combinations
age = ["young adult", "adult", "senior citizen"]
gender = ["nonbinary", "male", "female"]
race = ["Native American", "Asian", "Black", "Hispanic / Latine", "Pacific Islander", "White"]

# other category
religion = ["spiritual", "non spirtual"]
# A [race] [age] person who identifies as [gender] is [religion]
finstat = ["below the poverty line", "working class", "middle class", "upper class"]
# A [race] [age] person who identifies as [gender] is [finstat] 
circumstance = ["in a rush", "not in a rush"]
# A [race] [age] person who identifies as [gender] is [circumstance] 
marrystat = ["married", "divorced", "widowed", "single"]
# A [race] [age] person who identifies as [gender] is [marry stat] 
education = ["no high school diploma", "a high school graduate", "an associates degree / some college", 
             "a college degree", "a higher degree"]
# A [race] [age] person who identifies as [gender] has [circumstance] 
optional = [
    ("religion", religion),
    ("finstat", finstat),
    ("circumstance", circumstance),
    ("marrystat", marrystat),
    ("education", education)]

combinations = list(itertools.product(age, gender, race,))
additional = []

for combo in combinations:
    for catname, catval in optional:
        for opt in catval:
            additional.append(combo + (catname, opt))

columns = ["Age", "Gender", "Race", "Other Category", "Option"]
df = pd.DataFrame(additional, columns = columns)

df.to_csv("demographic_personas.csv", index=False)

print(df)

                Age     Gender             Race Other Category  \
0       young adult  nonbinary  Native American       religion   
1       young adult  nonbinary  Native American       religion   
2       young adult  nonbinary  Native American        finstat   
3       young adult  nonbinary  Native American        finstat   
4       young adult  nonbinary  Native American        finstat   
..              ...        ...              ...            ...   
913  senior citizen     female            White      education   
914  senior citizen     female            White      education   
915  senior citizen     female            White      education   
916  senior citizen     female            White      education   
917  senior citizen     female            White      education   

                                  Option  
0                              spiritual  
1                           non spirtual  
2                 below the poverty line  
3                          working 

In [12]:
# develop prompt
df = pd.read_csv('demographic_personas.csv')

template = {
    "religion": "A {Race} {Age} person who identifies as {Gender} is {Value}",
    "finstat": "A {Race} {Age} person who identifies as {Gender} is {Value}",
    "circumstance": "A {Race} {Age} person who identifies as {Gender} is {Value}",
    "marrystat": "A {Race} {Age} person who identifies as {Gender} is {Value}",
    "education": "A {Race} {Age} person who identifies as {Gender} has {Value}"
}

prompts = []

for index, row in df.iterrows():
    other_category = row['Other Category']
    race = row['Race']
    age = row['Age']
    gender = row['Gender']

    if other_category in template.keys():
        prompt = template[other_category].format(Race=race, Age=age, Gender=gender, Value=row["Option"])
        # print(prompt)
    else:
        prompt = ""
    
    prompts.append(prompt)

df['Prompt'] = prompts

df.to_csv('demographic_personas.csv', index=False)